In [3]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

Time-dependent Poisson equation
-------------------------------------
Let consider again the 1D stick held at a fixed temperature $u=1$ at one end, and $u=0$ at the other. We are interested in observing the variation of temperature both in time and in space for $t=(0,T],$ where $T$ . Identifying the rod with the interval $[0, 1]$, we let $u(x, t)$ denote the temperature of the rod at the point $x$ and time $t$. In this case the temperature $u$ depends both on time and space. The equation that models this phenomenon can be written as  $$\frac{\partial u}{\partial t} - \frac{\partial^2u} {\partial x^2} = 0$$ with the boundary conditions $u=1$ at $x=0$, and $u=0$ at $x=1$. As we have done for the stationary Poisson equation, we will discretize the equation using the Finite Element Method in space. In addition to the boundary conditions we should provide an initial condition and define a discretization method for the time derivative. 

 <img src="heat.jpg" alt="Heated rod" style="width: 300px;"/>

Time discretization
----------------
We consider $M$ points uniformily in the time interval $(0,T].$ The first point in time is $t^0 = 0$ and the last point in time is $t^M = T.$ The distance between the points in time is $\Delta t = \frac{T}{M-1}.$ The time derivative can be discretized using the implicit Euler method: $$\frac{\partial u(t^m)}{\partial t} \approx \frac{u^m - u^{m-1}}{\Delta t},$$ where $u^m = u(t^m).$ From this scheme it is clear that we need an initial condition for $t = t^0.$ Given an initial condition such that $u(t^0) = u_0,$ the semi-discretization of the equation reads as: $$u^m - \Delta t \Delta u^m = u^{m-1},$$ for $m = 1, \dots, M.$ 


The weak form
------------------------

To illustrate, take your equation and multiply it by a new function v called a test function, giving you $v\cdot u_{xx} = 0$. Then, integrate over the whole domain, giving you the equation $$\int \limits_0^1 v \cdot u_{xx}  \text{dx} = 0$$ 

In some sense this is 'easier' to satisfy than the original equation, because it is possible that $\int \limits_0^1 f  \:
\text{dx} = 0$ even though $f \neq 0$ at every point.However, it is possible to show that, roughly speaking, if the integral equation is true for _every_ function $v$, then the original equation must be true as well, so requiring the latter equation to hold for _all_ v gives us something equivalent to the original equation. (For an idea of why this is true, think about what happens when v is a very narrow 'bump' function which is approximately 1 near some point, 
and rapidly decays to 0 outside that point.)

Now, this equation still involves second derivatives of u. However, using integration by parts, we can "move" some derivatives to v as well. Because $$(v \cdot u_x)_x = v_x \cdot u_x + v \cdot u_{xx}$$
we get the equation $$0 = \int\limits_0^1 v \cdot u_{xx} \: \text{dx}= \int\limits_0^1 (v \cdot u_x)_x \: \text{dx} - \int\limits_0^1 v_x \cdot u_x \: \text{dx} = v(1)u_x(1) - v(0) u_x(0) - \int\limits_0^1 v_x \cdot u_x \: \text{dx}$$

And this is an equation with only first derivatives!
While the derivative has a jump discontinuity at the nodes, this is not a problem because the value of a function at a single point doesn't affect its integral. So to summarize, we have shown that our original equation $u_{xx} = 0$ is in some sense equivalent to finding a $u$ which satisfies $$v(1)u_x(1) - v(0) u_x(0) - \int\limits_0^1 v_x \cdot u_x \: \text{dx}= 0$$ for all functions $v$. 
We call the original equation the _strong form_, and the new equation the _weak form_. 

Building a system of linear equations from the weak form
----------------------------------------------------------------
Now we can carry through our original plan of setting $u = \sum_i u_i \phi_i$ and building a system of equations to solve for the real numbers u_i. First, note that it is very easy to find $u_0$ and $u_5$. Because each $\phi_i$ is 1 at the mesh point $x_i$ and $0$ otherwise, setting $x=0$ in $u(x) = \sum_i u_i \phi_i(x)$ yields that
$1 = u(0) = u_0$, and setting $x=1$ similarly yields that $0 = \phi_5 = u_5$. So we need 4 more equations to find $u_2, u_3, u_4, u_5$.

Putting $v = \phi_j$ for $j = 2, 3, 4 or 5$, we see that $v(0) = v(1) = 0$, so our equation becomes $$- \int\limits_0^1(\phi_j)_x \cdot \sum_i u_i (\phi_i)_x = 0$$ or equivalently $$\sum_i u_i \int\limits_0^1 (\phi_i)_x \cdot (\phi_j)_x = 0$$

Because $\phi_i$, $\phi_j$ are known functions, we see that if we first compute the quantities $\int\limits_0^1((\phi_i)_x \cdot (\phi_j)_x = 0$ for all pairs $i,j$, this will be a nice system of linear equations
in good old real numbers which we can solve. 

We also see that $(\phi_i)_x \cdot (\phi_j)_x$ is fairly easy to compute  - recalling what $\phi_i$ looks like, we see that $(\phi_i)_x$ is $+\frac 1 h$ between $x_{i-1}$ and $x_i$, $-\frac 1 h$ between $x_i$ and $x_{i+1}$, and $0$ otherwise.
So $\int\limits_0^1 (\phi_i)_x \cdot (\phi_j)_x \: \text{dx}$ is:

* $\frac 2 h$ when $i=j$ 
* $-\frac 1 h$ when $i = j \pm 1$ 
* $0$ when $i, j$ are more than 1 apart. 

If this is unclear, scroll up a few cells to take a look at the graphs of $(\phi_i)_x$.


To illustrate, the equation we get from putting $v=\phi_2$, then, looks like 
$$-\frac 1 h \cdot u_1 + \frac 2 h \cdot u_2 -\frac 1 h \cdot u_3 = 0$$ Putting $v=\phi_3$ we get $$-\frac 1 h \cdot u_2 + \frac 2 h \cdot u_3 - \frac 1 h \cdot u_4 = 0$$ and so on. 

Notice that this system is *sparse* in the sense that the equation you get for $v=\phi_i$ only has 3 unknowns: $u_{i-1}, u_i$ and $u_{i+1}$, and that this would be true even if $N$ was a much larger number than 6. This sparsity comes from the fact that the $\phi_i$ are very 'local' functions. Though we will not go into details here, this locality is really what makes the finite element method so useful, as a sparse system of equations is a lot easier to solve numerically than a dense one. Writing our system in matrix form, we get:


 $$\begin{bmatrix}
    1 & 0 & 0 & 0 & 0 & 0 \\
    -\frac 1 h & \frac 2h & - \frac 1h & 0 & 0 & 0\\
    0 & -\frac 1 h & \frac 2h & - \frac 1h & 0 & 0 \\
    0 & 0 & -\frac 1 h & \frac 2h & - \frac 1h & 0 \\
    0 & 0 & 0 & -\frac 1 h & \frac 2h & - \frac 1h \\
    0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}\begin{bmatrix}
    u_0 \\[2.2pt]
    u_1 \\[2.2pt]
    u_2 \\[2.2pt]
    u_3 \\[2.2pt]
    u_4 \\[2.2pt]
    u_5
\end{bmatrix} = \begin{bmatrix}
    1 \\[2.2pt]
    0 \\[2.2pt]
    0 \\[2.2pt]
    0 \\[2.2pt]
    0 \\[2.2pt]
    0
\end{bmatrix}$$

Feel free to multiply out the above to see that we in fact get the equations we wrote down previously. Note also that the first and last rows are special, corresponding to us having boundary conditions at the corresponding nodes $x_0=0$ and $x_5=1$.


Solving the system of linear equations
-------------------------------------------
In any case, this can now be packaged up and handed over to whichever linear algebra software you prefer.
Then, once the $u_i$ have been found, our approximation of the solution u is then the function $\sum_i u_i \phi_i(x)$. Below, we do this in numpy:


In [4]:
H = 1/h
A = np.array(([
    [1, 0, 0, 0, 0, 0],
    [-H, 2*H, -H, 0, 0, 0],
    [0, -H, 2*H, -H, 0, 0],
    [0, 0, -H, 2*H, -H, 0],
    [0, 0, 0, -H, 2*H, -H],
    [0, 0, 0, 0, 0, 1],
    
]))
b = np.array([1, 0, 0, 0, 0, 0])

u = np.linalg.solve(A, b)
print u

NameError: name 'h' is not defined

So the solution is $$u = \begin{bmatrix}
    1 \\
    0.8 \\
    0.6 \\
    0.4 \\
    0.2 \\
    0
\end{bmatrix}$$

To see what this looks like as a function of $x$, we recall that $u_i$ were defined so that $u(x) = \sum_i u_i \phi_i(x)$. Substituting in our solution $u_i$, we see that $u$ looks like the below:



In [ ]:
U = np.zeros((N))
for i in range(N):
    U += u[i]*phi(i)
    
plt.plot(x, [0]*N, "ro", label="mesh points")
plt.plot(x, U, "b-", label="numerical solution")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend();

We see that in this case we get a very nice and linear solution $u(x) = 1-x$. Because $u_{xx}=0$ and the boundary conditions are satisfied, this is actually the real solution of the equation.

Summary
--------
To recap the method used:

1. Take the original equation, multiply it by a test function v and integrate by parts to get rid of all second derivatives
2. Choose some basis functions $\phi_i$, write $u=\sum_i u_i \phi_i$ and put $v = \phi_0, \phi_1, \phi_2$ successively to get a bunch of equations
3. Express the resulting equations in terms of known stuff with $\phi_i$, build a linear system of equations and solve it for the unknowns $u_i$
4. The solution $u(x)$ is then approximated as $\sum_i u_i \phi_i(x)$

This is the finite element method. It can also be used to solve more complicated PDEs, handle different basis functions $\phi_i$, work with more complicated domains $\Omega$ and more, without significant deviations from the above list. In future lectures, we will give examples of the method applied to more exciting equations and demonstrate how to use *FEniCS*, a software suite.


Homework
---------------------

Repeat the procedure outlined here for the equation $-u_{xx} + u = 0$.

 # Brief summaries (intended for presentation view only)

<br /> <br /> <br /> <br /> <br /> <br />  <br /> <br /> <br /> <br /> <br />  
<div style="text-align: center"> <font size="6"> <strong> Introduction to the Finite Element Method </strong> </font></div> 

The Poisson equation
-----------------------
* Model of heat conduction:  $$-\frac{\partial^2u} {\partial x^2} = 0$$
* Domain: 1D rod identified with the interval $[0, 1]$, $u(x) = $ temperature at point $x$
* Boundary conditions: rod held at fixed temperatures $u(0)=1$, $u(1) = 0$ at ends
* Notation: $u_x = $ derivative, $u_{xx}=$ second derivative

Discretization
----------------
* Choose $N=6$ points uniformly in $[0, 1]$ to get a mesh with nodes $x_0, \: \ldots, \: x_5$
* $V_h = $ space of piecewise linear functions with break points at mesh points 
* $h = \frac 1 5$ length of subinterval
* Idea: maybe the solution $u$ isn't actually in $V_h$, but find best approximation there
* Define basis functions $\phi_i$ which are "hats" centered at $x_i$

Why we need the weak form
-----------------------------
* **Strategy:** Write $u=u_0 \phi_0 + \ldots + u_5 \phi_5$, insert into $u_{xx}=0$ and solve for coefficients $u_i$ 
* **Problem:** Second derivatives of $\phi_i$ are weird
* **Solution:** Replace $u_{xx}=0$ by 'equivalent' equation *without* second derivatives (weak form)

Finding the weak form
------------------------
* Multiply by arbitrary function (test function) $v$ and integrate over $\Omega$: $$\int \limits_0^1 v \cdot u_{xx}  \text{dx} \:=\: 0$$ 
* Integrate by parts: $$(v \cdot u_x)_x = v_x \cdot u_x + v \cdot u_{xx}$$

* Rewrite integral: 
 $$\int\limits_0^1 v \cdot u_{xx} \: \text{dx} \:=\: v(1)u_x(1) \:-\: v(0) u_x(0) \:-\: \int\limits_0^1 v_x \cdot u_x \: \text{dx} $$
 
* Conclusion:
  $$u_{xx} = 0 \hspace{4mm} "\Longleftrightarrow" \hspace{4mm}  v(1)u_x(1) \:-\: v(0) u_x(0) \:-\: \int\limits_0^1 v_x \cdot u_x \: \text{dx} = 0 \hspace{5mm} \text{ for all } v$$

Building a system of linear equations from the weak form
----------------------------------------------------------------
* Put $u(x) = \sum_i u_i \phi_i(x)$ into weak form $v(1)u_x(1) - v(0) u_x(0) - \int\limits_0^1 v_x \cdot u_x \: 
\text{dx}= 0$ to get equation with unknowns $u_i$ (numbers) instead of $u$ (function)
* Boundary conditions: $u(0) = 1 = u_0$, and $u(1) = 0 = u_5$
* Put $v = \phi_j$ into weak form to get $$- \int\limits_0^1(\phi_j)_x \cdot \sum_i u_i (\phi_i)_x = 0$$
* Because $\int \sum = \sum \int$, rewrite as $$\sum_i u_i \int\limits_0^1 (\phi_i)_x \cdot (\phi_j)_x = 0$$
* Compute $\int\limits_0^1 (\phi_i)_x \cdot (\phi_j)_x$ to get $\frac 2h, -\frac 1 h$ or $0$ depending on $i-j$
* Put appropriate values of $\int\limits_0^1 (\phi_i)_x \cdot (\phi_j)_x$ into equation to get linear system of equations for $u_j$: $$-\frac 1 h \cdot u_{j-1} + \frac 2 h \cdot u_j -\frac 1 h \cdot u_{j+1} = 0$$

 Solving the system of linear equations
 -------------------------------------------
 $Au=b$, where
 
 $$A = \begin{bmatrix}
    1 & 0 & 0 & 0 & 0 & 0 \\
    -\frac 1 h & \frac 2h & - \frac 1h & 0 & 0 & 0\\
    0 & -\frac 1 h & \frac 2h & - \frac 1h & 0 & 0 \\
    0 & 0 & -\frac 1 h & \frac 2h & - \frac 1h & 0 \\
    0 & 0 & 0 & -\frac 1 h & \frac 2h & - \frac 1h \\
    0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}, \: u = \begin{bmatrix}
    u_0 \\[2.2pt]
    u_1 \\[2.2pt]
    u_2 \\[2.2pt]
    u_3 \\[2.2pt]
    u_4 \\[2.2pt]
    u_5
\end{bmatrix}, \: b = \begin{bmatrix}
    1 \\[2.2pt]
    0 \\[2.2pt]
    0 \\[2.2pt]
    0 \\[2.2pt]
    0 \\[2.2pt]
    0
\end{bmatrix}$$


Solution: $$u = \begin{bmatrix}
    1 \\
    0.8 \\
    0.6 \\
    0.4 \\
    0.2 \\
    0
\end{bmatrix}$$
So the solution of our original equation is $$u \: = \: 1 \cdot \phi_0 \: + \: 0.8 \cdot \phi_1 \: + \: 0.6 \cdot \phi_2 \: + \: 0.4 \cdot \phi_3 \: + \: 0.2 \cdot \phi_4 \: + \: 0 \cdot \phi_5$$ which actually works out to 

$$u(x) = 1 - x$$